In [ ]:
# %pip install --upgrade openai

# ** Step 1 ** 

### Import Library

In [52]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [53]:
load_dotenv()

python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 23
python-dotenv could not parse statement starting at line 24
python-dotenv could not parse statement starting at line 26
python-dotenv could not parse statement starting at line 27
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 29
python-dotenv could not parse statement starting at line 31
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33
python-dotenv could not parse statement starting at line 34
python-dotenv could not parse statement starting at line 35
python-dotenv could not parse statement starting at line 36
python-dotenv could not parse statement starting at line 37
python-dotenv could not parse statement starting at line 39
python-dotenv could not parse statement starting at line 40
python-dotenv could not parse statement s

True

### 환경변수 설정

In [54]:
""" 환경변수 설정 """
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
assistant_id = os.getenv('ASSISTANT_ID')
vector_store_id = os.getenv('VECTOR_STORE_ID')

In [96]:
base_prompt="""당신은 재난 대응 전문가이자 온톨로지 엔지니어입니다.

            ## 중요: 일관된 출력을 위한 규칙
            - **반드시 동일한 순서로 클래스를 생성하세요**
            - **반드시 동일한 네이밍 규칙을 적용하세요**
            - **반드시 동일한 계층 구조를 유지하세요**

            ## 클래스 생성 순서 (절대 변경 금지)
            1. 최상위 클래스: ex:위기경보수준별조치사항
            2. 1단계: ex:위기경보수준별조치사항_1_관심
            3. 2단계: ex:위기경보수준별조치사항_2_주의
            4. 3단계: ex:위기경보수준별조치사항_3_경계
            5. 4단계: ex:위기경보수준별조치사항_4_심각
            6. 각 단계별 하위 클래스들을 동일한 순서로 생성


            [출력 목표]
            - Protege/GraphDB에서 즉시 파싱 가능한 **단일 TTL 블록**을 출력합니다.
            - 결과 형태는 아래 "필수 Prefix"와 "출력 구성·순서"를 **정확히 준수**합니다.
            - 제공 근거가 불충분한 항목은 생성하지 않고, "스킵 사유"를 최하단 요약 JSON(report)로만 남깁니다.

            [비파괴(append-only) 원칙]
            - 기존 TTL의 요소(클래스/속성/트리플)는 **변경·삭제 금지**.
            - 누락 시에만 **새로 추가**합니다.
            - 중복 후보 발견 시, **동일 IRI**면 중복 추가 금지, **상이한 IRI**지만 동일 개념으로 판단되면 추가하지 않고 report에만 기록합니다.

            [필수 Prefix]  ⬅ 반드시 이 순서·철자·공백으로 선언
            @prefix ex:      <http://example.org/disaster#> .
            @prefix rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
            @prefix rdfs:    <http://www.w3.org/2000/01/rdf-schema#> .
            @prefix owl:     <http://www.w3.org/2002/07/owl#> .
            @prefix dcterms: <http://purl.org/dc/terms/> .
            @prefix xsd:     <http://www.w3.org/2001/XMLSchema#> .

            [네이밍 규칙] (절대 변경 금지)
            - 형식: ex:위기경보수준별조치사항_[번호]_[수준]_[하위항목]
            - 예시: ex:위기경보수준별조치사항_1_관심_가_상황
            - 예시: ex:위기경보수준별조치사항_2_주의_나_조치목록
            - 리소스 IRI(local name)는 **영문/숫자/언더스코어(_)만** 사용.
            - 한국어 명칭은 **rdfs:label "…"@ko**로만 표기.
            - 기존 TTL에 이미 한글 로컬네임이 존재할 수 있음 → **기존 것은 유지**, 신규 생성 시에는 영문 IRI만 사용.
            - 동일 개념이 한글/영문 IRI로 이중 존재할 경우, **owl:equivalentClass 금지**, 대신 `rdfs:seeAlso`만 연결.

            [페이지·출처 메타데이터(강제)]
            - 새로 추가하는 **모든 리소스**에는 다음 메타데이터를 부여:
            - `dcterms:coverage`: "page_XXX"
            - `dcterms:source`: 문서/기관명
            - 가능하면 `ex:페이지정보`, `ex:섹션정보`
            - 근거 불명확 시 생성 금지.

            [관계/프로퍼티 추가 원칙]
            - 오브젝트/데이터 프로퍼티는 **문서에 명시된 관계/속성만** 추가.
            - Placeholder/예시 값(“00월 00일 …”) 금지.
            - 도메인/레인지/타입으로 참조된 클래스·속성이 없으면 반드시 append-only로 정의 추가.

            [출력 구성·순서]
            1) ###### 주석 헤더(섹션 구분)
            2) 클래스/인스턴스 정의 블록
            3) AnnotationProperty 정의 블록 (ex:페이지정보, ex:섹션정보)
            4) (선택) report JSON 블록:
            ```json
            {
            "added_classes": [],
            "added_instances": [],
            "added_properties": [],
            "skipped_as_duplicate": [],
            "skipped_due_to_insufficient_evidence": []
            }
            
            [형식 엄수·가드레일]
            - **코드 블록 한 개** 안에 **TTL만** 출력(리포트 JSON 포함 시 TTL 블록 종료 후 별도 코드 블록으로 출력).
            - Turtle 주석(`# ...`) 외 불필요한 설명, Markdown 텍스트 금지.
            - PDF 원문 인용 시 **25단어 이내**로 요약하여 rdfs:comment에만 기재.
            - 예시에 보이는 주석 태그(예: `# :contentReference[...]`)는 **입력에서 해당 정보가 존재할 때만** 동일 형식으로 주석에 덧붙여도 됨. 없으면 생략.
            - 파싱 오류를 유발하는 문법(세미콜론/마침표 누락, 따옴표 불일치, 잘못된 prefix) **절대 금지**.
            - 생성 중 애매하거나 근거 부족하면 **생성하지 말고** report JSON의 `skipped_due_to_insufficient_evidence`에 사유와 대상 후보를 기록.

            [검증 체크리스트(내장)]
            - Prefix 선언 순서·철자 일치 여부 확인.
            - 모든 신규 리소스에 rdfs:label(@ko) 존재 확인.
            - 새로 추가한 항목에 메타데이터(dcterms:coverage/source 또는 ex:페이지정보/섹션정보) 존재 확인.
            - 상위→하위 rdfs:subClassOf 계층 검증.
            - 중복 IRI/트리플 여부 검사 및 제거(append-only 원칙 준수).
            - 최종 TTL 파싱 가능성(문장부호·따옴표·언어태그) 점검.

            [실행 규율]
            - 본 BASE PROMPT는 공통 원칙만 명시합니다. **이번 작업의 범위/제한/우선순위는 호출 시 추가되는 Task Addendum(message_content)에 따릅니다.**
            - Addendum이 “클래스 생성만”이라면 인스턴스/프로퍼티는 **추가하지 않습니다**(반대의 경우도 동일).
            - 항상 입력 근거(문서/조각 JSON)에 **정확히 합치**하는 결과만 출력합니다."""

class_create_prompt="""
                        목차를 기준으로 다른 문서들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요. 

                        중요: 일관된 결과를 위해 다음을 반드시 준수하세요:
                        1. 클래스 생성 순서를 절대 변경하지 마세요
                        2. 네이밍 규칙을 절대 변경하지 마세요  
                        3. 계층 구조를 절대 변경하지 마세요
                        4. 동일한 패턴으로 모든 클래스를 생성하세요

                        TTL 파일은 Turtle 형식으로 작성하고, 적절한 네임스페이스와 클래스, 속성을 정의해주세요.
                        결과물은 한 번에 출력해주세요.
                    """

addundum_page_json_prompt="""
                            당신은 PDF 문서에서 문단 단위의 구조를 분석하여 JSON으로 출력하는 전문가입니다.  
                            다음 규칙을 반드시 지켜 JSON을 생성하세요.  

                            ---

                            ## 출력 형식 (JSON 배열)  
                            각 항목은 하나의 "문단 단위 조각(fragment)"입니다.  
                            - **section_uid**: 단락을 식별할 수 있는 고유 ID (예: III-3-A, III-3-B-1 등 목차 기반 ID)  
                            - **page**: 해당 조각이 속한 페이지 번호 (정수)  
                            - **intra_page_index**: 같은 페이지 안에서 문단이 여러 개일 경우 순서를 나타내는 번호 (1부터 시작)  
                            - **title**: 문단/단락의 제목 전체  
                            - **hierarchy_path**: 상위 목차 계층을 배열로 표시  

                            ---

                            ## 규칙

                            1. 항상 `page` 단위로만 출력한다.  
                            - `startpage`, `endpage` 같은 범위 값은 절대 사용하지 않는다.  
                            - 하나의 JSON 객체 = 정확히 한 페이지.  

                            2. 같은 페이지 안에서 여러 단락이 있으면  
                            - 각각 별도의 JSON 객체를 만들고, `intra_page_index`는 순차적으로 1부터 증가한다.  

                            3. 여러 페이지에 걸친 단락은  
                            - 페이지마다 별도의 JSON 객체를 만든다.  
                            - `section_uid`는 동일하게 유지한다.  
                            - 첫 페이지에는 원래 제목을 그대로 사용한다.  
                            - 두 번째 페이지 이후에는 `"(계속)"`을 제목 뒤에 붙여 표시한다.  

                            4. 중첩 및 누락 금지  
                            - 같은 페이지 번호가 겹치는 범위를 절대 쓰지 말 것.  
                            - 각 페이지에서 해당 단락의 일부라도 존재하면 반드시 JSON 객체를 만든다.  

                            5. 출력 방식  
                            - 출력은 반드시 JSON 배열 하나만 내놓는다.  
                            - 불필요한 설명, 주석, 텍스트는 절대 추가하지 않는다.  

                            6. 반드시 누락없이 모든 페이지를 확인한다.

                            ---

                            ## json 예시

                            [
                            {
                                "section_uid": "III-3-B-1",
                                "page": 63,
                                "intra_page_index": 2,
                                "title": "Ⅲ. 위기경보 수준별 조치사항 - 3 경계 경보 - 나. 조치목록 및 내용 - 1) 조치목록",
                                "hierarchy_path": ["Ⅲ 위기경보 수준별 조치사항", "3 경계 경보", "나. 조치목록 및 내용", "1) 조치목록"]
                            },
                            {
                                "section_uid": "III-3-B-1",
                                "page": 64,
                                "intra_page_index": 1,
                                "title": "Ⅲ. 위기경보 수준별 조치사항 - 3 경계 경보 - 나. 조치목록 및 내용 - 1) 조치목록 (계속)",
                                "hierarchy_path": ["Ⅲ 위기경보 수준별 조치사항", "3 경계 경보", "나. 조치목록 및 내용", "1) 조치목록"]
                            }
                            ]

                            **중요** 이 프롬프트를 사용할 때는 TTL 형식이 아닌 반드시 JSON 형식으로만 출력합니다.
                        """


add_subclass_prompt = """
                        [TASK ADDENDUM — STEP 3: SUBCLASS COMPLETION (계층 보완 전용)]

                        이번 작업은 **1단계에서 생성된 클래스 계층(최상위~하위)**을, PDF/목차/페이지-조각(JSON) 근거로 **누락된 서브클래스만 추가(append-only)** 하는 데 한정합니다.
                        - **인스턴스 생성 금지**, **오브젝트/데이터/애노테이션 프로퍼티 신규 정의 금지**.
                        - 단, 새로 추가되는 **모든 클래스**에는 라벨/페이지/출처 메타를 **반드시** 부여합니다.

                        [입력]
                        - (필수) 1단계 산출 TTL (기존 계층)
                        - (필수) PDF 원문/텍스트, 목차.txt
                        - (권장) 페이지-조각(JSON: page, section_uid, title, hierarchy_path, intra_page_index)

                        [보완 대상·범위]
                        1) 최상위 루트(`ex:위기경보수준별조치사항` a owl:Class)의 하위로,
                        - **1~4 단계(관심/주의/경계/심각)** 클래스가 존재해야 함.
                        2) 각 단계의 하위에 다음 서브트리(모두 **클래스**)를 완비:
                        - `…_가_상황`
                        - `…_나_조치목록및내용` → 하위로
                            - `…_나_1_조치목록`
                            - `…_나_2_조치내용` → 하위로
                            - `…_나_2_1_경보(및)상황접수`
                            - `…_나_2_2_보고및전파`
                            - `…_나_2_3_조치`
                        3) 문서(혹은 페이지-조각 JSON)에 “조치내용의 세부 항목(예: 감시활동강화, 안전실태점검…)”이 **명시**되어 있으면,
                        - 해당 항목을 **클래스(조치내용의 하위)** 로 추가하되, 근거 페이지/출처를 부여.
                        - 문서 표현이 “개별 행위/사건/리스트 항목(실행 인스턴스에 가까움)”일 경우 **이번 단계에서는 추가 금지**하고 report에 “instance 후보”로 기록.

                        [네이밍 규칙(강화)]
                        - **신규 로컬네임은 영문/숫자/언더스코어만** 사용. (기존 한글 IRI는 유지 가능)
                        - 기본 패턴(경로키 기반; 예시):
                        - 단계: `ex:Alert_1_Attention`
                        - 상황: `ex:Alert_1_Attention___A_Situation`
                        - 조치목록및내용: `ex:Alert_1_Attention___B_ActionSet`
                        - 조치목록: `ex:Alert_1_Attention___B1_ActionList`
                        - 조치내용: `ex:Alert_1_Attention___B2_ActionItems`
                        - 조치내용 하위 항목(예: 감시활동강화): `ex:Alert_1_Attention___B2_GamSiHwaldong_Ganga` (의미 보존 로마자/영문화)
                        - 단, **이미 존재하는 한글 IRI는 변경/삭제 금지**. 동일 개념을 영문 IRI로 보완 생성할 필요가 있을 때는 **rdfs:seeAlso**로만 연결(동등 선언 금지).

                        [메타데이터(강제)]
                        - 새로 추가되는 **모든 클래스**에 아래 중 최소 2개 이상:
                        - `rdfs:label "…"@ko`
                        - `dcterms:coverage "page_XXX"` (페이지 번호를 3자리로 zero-padding)
                        - `dcterms:source "문서/기관명 또는 식별 문자열"`
                        - (있으면) `ex:페이지정보 "XXX"`, `ex:섹션정보 "section_uid"`
                        - rdfs:comment 사용 시 **최대 25단어**로 요약.

                        [계층 연결 규칙]
                        - 모든 신규 클래스는 반드시 상위 개념에 `rdfs:subClassOf`로 연결.
                        - 상→하 일관성: 단계 → (가)상황, (나)조치목록및내용 → (1)조치목록, (2)조치내용 → (2-1/2/3 및 문서에 명시된 세부 항목).

                        [중복/충돌 처리]
                        - **동일 IRI**가 이미 있으면 새로 만들지 않음.
                        - **동일 개념 의심(이명/동일 라벨)**인데 IRI가 다르면 **추가 생성 금지**하고 report에 `skipped_as_duplicate`로 양쪽 IRI와 근거 요약 기록.
                        - 기존 한글 IRI와 신규 영문 IRI가 같은 개념이면, **이번 단계에서 owl:equivalentClass/상속 금지**, `rdfs:seeAlso`만 연결.

                        [증거·추정 금지]
                        - PDF/목차/페이지-조각(JSON) **명시 사항만** 반영.
                        - Placeholder/예시/추정 생성 금지. 근거 없으면 report의 `skipped_due_to_insufficient_evidence`에 사유 기록.

                        [출력 형식(단일 TTL 블록)]
                        - BASE_PROMPT의 **필수 Prefix 순서**를 사용하여 맨 앞에 1회 선언.
                        - **클래스 정의만** 출력(신규/보완분 중심). 이미 있는 정의를 다시 재정의하지 않음(append-only).
                        - 블록 하단에 AnnotationProperty 정의(없으면 스킵 가능), 마지막에 **report JSON** 1개 블록을 별도 코드블록으로 출력.

                        [생성 예시(형태 가이드; 실제 값은 입력 근거로 채움)]
                        - (한글 IRI가 이미 존재하는 경우) 그 IRI를 상위로 두고, 신규 영문 클래스 추가 시:
                        ex:Alert_1_Attention___B2_ActionItems a owl:Class ;
                            rdfs:subClassOf ex:위기경보수준별조치사항_1_관심_나_2_조치내용 ;
                            rdfs:label "조치내용(관심)"@ko ;
                            dcterms:coverage "page_052" ;
                            dcterms:source "LH_풍수해 행동매뉴얼_ver1.0_2024" ;
                            rdfs:seeAlso ex:위기경보수준별조치사항_1_관심_나_2_조치내용 .

                        - (세부 항목) 감시활동강화:
                        ex:Alert_1_Attention___B2_Surveillance_Enhancement a owl:Class ;
                            rdfs:subClassOf ex:위기경보수준별조치사항_1_관심_나_2_조치내용 ;
                            rdfs:label "감시활동강화"@ko ;
                            dcterms:coverage "page_051" ;
                            dcterms:source "LH_풍수해 행동매뉴얼_ver1.0_2024" .

                        [검증 체크리스트(자동 적용)]
                        - Prefix 순서/철자 일치.
                        - 새로 추가한 모든 클래스에 rdfs:label(@ko) + 페이지/출처 메타(최소 2개) 존재.
                        - 상→하 rdfs:subClassOf 계층의 단절/역참조 없음.
                        - owl:Class/rdfs:Class 혼용 시, 신규는 **owl:Class**로 통일.
                        - Placeholder/추정 생성 없음.
                        - 한글/영문 IRI 중복 시 seeAlso만 연결(equivalentClass 금지).

                        [출력 종료 후 report JSON 예시]
                        {
                        "added_classes": ["ex:Alert_1_Attention___B2_Surveillance_Enhancement", "..."],
                        "added_instances": [],
                        "added_properties": [],
                        "skipped_as_duplicate": [
                            {"candidate":"ex:Alert_1_Attention___B2_ActionItems","reason":"이미 존재하는 한글 IRI와 동일 개념 의심","existing":"ex:위기경보수준별조치사항_1_관심_나_2_조치내용"}
                        ],
                        "skipped_due_to_insufficient_evidence": [
                            {"candidate":"ex:Alert_1_Attention___B2_Field_Evacuation_Operation","reason":"문서에 클래스화 가능한 항목으로 명시 없음(인스턴스 후보)"}
                        ]
                        }

                        """

In [56]:
def print_status(message, status="INFO"):
    """ 터미널 메세지 커스텀(선택)"""
    status_color = {
        "INFO": "\033[94m", # blue
        "SUCCESS": "\033[92m", # green
        "WARING": "\033[93m", # yellow
        "ERROR": "\003[91m", # red
        "RESET": "\033[0m" # reset
    }

    color = status_color.get(status, status_color["INFO"])
    reset = status_color["RESET"]

    print(f"{color}[{status}] {message}{reset}")

### 파일 업로드 후 벡터 스토어에 추가 O

In [57]:
def upload_file_to_vector_store(file_paths):
    # 파일 업로드
    uploaded_file_ids = []

    for file_path in file_paths:
        if os.path.exists(file_path):
            try :
                with open(file_path, 'rb') as f :
                    uploaded_file = client.files.create(
                        file=f,
                        purpose='assistants'
                    )

                print_status(f"파일 업로드 완료: {uploaded_file.id}", "SUCCESS")


                # 벡터 스토어에 추가
                client.vector_stores.files.create(
                    vector_store_id = vector_store_id,
                    file_id = uploaded_file.id
                )

                print_status(f"벡터 스토어에 추가 완료: {uploaded_file.id}", "SUCCESS")
                uploaded_file_ids.append(uploaded_file.id)


            except Exception as e :
                print_status(f"파일을 찾을 수 없습니다: {file_path}", "ERROR")
                continue
    #업로드된 파일 개수 확인
    print_status(f"총 {len(uploaded_file_ids)}개의 파일이 벡터 스토어에 업로드 되었습니다", "SUCCESS")
    return uploaded_file_ids


# 로컬에 있는 3장 pdf 파일과 목차 경로 정의
from pathlib import Path

chat3_index_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\Ontology_Root\목차.txt")
actions_level_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\\Ontology_Root\위기경보수준별조치사항.pdf")
docs_path = [chat3_index_docs_path, actions_level_docs_path]

uploaded_file_ids = upload_file_to_vector_store(docs_path)

[SUCCESS] 파일 업로드 완료: file-2rPod6Wc8598xF978E8u7N
[SUCCESS] 벡터 스토어에 추가 완료: file-2rPod6Wc8598xF978E8u7N
[SUCCESS] 파일 업로드 완료: file-9exKqQbzoew6ri9Jmnfz8L
[SUCCESS] 벡터 스토어에 추가 완료: file-9exKqQbzoew6ri9Jmnfz8L
[SUCCESS] 총 2개의 파일이 벡터 스토어에 업로드 되었습니다


### 스레드 생성 및 메세지 추가 완료 O

In [58]:
def create_thread(client=client):
    thread = client.beta.threads.create()
    print_status(f"스레드 생성 완료: {thread.id}", "SUCCESS")

    thread_id = thread.id
    return thread.id

thread_id = create_thread()

C:\Users\user\AppData\Local\Temp\ipykernel_55848\103296605.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


[SUCCESS] 스레드 생성 완료: thread_EVui26HzfUynqMI7X6udLPyI


In [59]:
def add_message_to_thread(client, thread_id, base_prompt, uploaded_file_ids):
    message_content = base_prompt + "\n\n 첨부된 파일들을 분석하여 TTL 파일을 생성해주세요. 생성 후에는 첨부된 PDF 문서를 한 페이지씩 읽어서, subclass를 보완하여 추가해주세요. 이 때 기존의 요소들은 변경하지 않고 유지한 상태로 추가만 진행합니다."

    thread_message = client.beta.threads.messages.create(
        thread_id=thread_id,\
        role = "user",
        content=message_content,
        attachments=[
            {
                "file_id": file_id,
                "tools": [{"type": "file_search"}]
            }

            for file_id in uploaded_file_ids
        ]
    )

    print_status(f"스레드 메세지 추가 완료", "SUCCESS")
    return thread_message.id

add_message_to_thread(client, thread_id, base_prompt, uploaded_file_ids)

C:\Users\user\AppData\Local\Temp\ipykernel_55848\3398931413.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread_message = client.beta.threads.messages.create(


[SUCCESS] 스레드 메세지 추가 완료


'msg_wBy8b7e1ylggXAj5arCwK8AC'

### Assistant 실행 O

run_status.status == 'queued'        # 대기 중  
run_status.status == 'in_progress'   # 실행 중  
run_status.status == 'completed'     # 완료  
run_status.status == 'failed'        # 실패  
run_status.status == 'cancelled'     # 취소됨  
run_status.status == 'expired'       # 만료됨  
run_status.status == 'requires_action'  # 추가 액션 필요

Assistatnt를 실행하면 추가 vector stores가 생성됨

In [71]:
import time

def run_assistant_with_task_prompt(client, thread_id, base_prompt, task_prompt):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions= task_prompt + base_prompt
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == 'completed':
            print_status("Assistant 실행 완료", "SUCCESS")
            break
        
        elif run_status.status == 'failed':
            print_status("Assistant 실행 실패", "ERROR")

        elif run_status.status == 'requires_action':
            print_status("추가 액션 필요", "WARNING")
        
        elif run_status.status == 'in_progress':
            print_status("Assistant 실행 중...", "INFO")

            time.sleep(10)

    message = client.beta.threads.messages.list(
        thread_id=thread_id,
        order="desc"
    )

    return message

ontology_message = run_assistant_with_task_prompt(client, thread_id, base_prompt, class_create_prompt)
print([str(ontology_message)][:10])


C:\Users\user\AppData\Local\Temp\ipykernel_55848\2521358251.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\user\AppData\Local\Temp\ipykernel_55848\2521358251.py:11: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[SUCCESS] Assistant 실행 완료


C:\Users\user\AppData\Local\Temp\ipykernel_55848\2521358251.py:31: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.list(


['SyncCursorPage[Message](data=[Message(id=\'msg_6G5TDl93icXZ2RDdPAOxlsN0\', assistant_id=\'asst_W7kaWMqWblyhhVj1rCP0mVow\', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value=\'```turtle\\n@prefix ex:      <http://example.org/disaster#> .\\n@prefix rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\\n@prefix rdfs:    <http://www.w3.org/2000/01/rdf-schema#> .\\n@prefix owl:     <http://www.w3.org/2002/07/owl#> .\\n@prefix dcterms: <http://purl.org/dc/terms/> .\\n@prefix xsd:     <http://www.w3.org/2001/XMLSchema#> .\\n\\n###### 위기경보수준별조치사항 최상위 클래스 정의\\n\\nex:위기경보수준별조치사항 a owl:Class ;\\n    rdfs:label "위기경보수준별조치사항"@ko ;\\n    dcterms:coverage "page_68-80" ;\\n    dcterms:source "위기경보수준별조치사항.pdf" .\\n\\n###### 1단계: 관심\\n\\nex:위기경보수준별조치사항_1_관심 a owl:Class ;\\n    rdfs:label "위기경보수준별조치사항 1 관심"@ko ;\\n    rdfs:subClassOf ex:위기경보수준별조치사항 ;\\n    dcterms:coverage "page_68-69" ;\\n    dcterms:source "위기경보수준별조치사항.pdf" .\\n\\n###### 2단계: 주의\\n\\nex:

### 응답 결과 TTL 추출 O

In [61]:
import re

def _validate_basic_ttl_syntax(ttl_content):
    """
    기본적인 TTL 문법 검증
    
    Args:
        ttl_content: TTL 내용 문자열
    
    Returns:
        bool: 기본 문법이 올바른지 여부
    """
    try:
        # 기본적인 TTL 요소들 확인
        has_prefix = '@prefix' in ttl_content or '@base' in ttl_content
        has_class = 'rdfs:Class' in ttl_content or 'owl:Class' in ttl_content
        has_proper_ending = ttl_content.strip().endswith('.')
        
        # 괄호 균형 확인
        open_brackets = ttl_content.count('[')
        close_brackets = ttl_content.count(']')
        bracket_balanced = open_brackets == close_brackets
        
        # 세미콜론과 마침표 확인
        has_semicolons = ';' in ttl_content
        has_periods = '.' in ttl_content
        
        validation_result = has_prefix and has_class and has_proper_ending and bracket_balanced
        
        if not validation_result:
            print_status("기본 TTL 문법 검증 실패", "WARNING")
            print_status(f"  - Prefix 선언: {has_prefix}", "INFO")
            print_status(f"  - Class 선언: {has_class}", "INFO")
            print_status(f"  - 올바른 종료: {has_proper_ending}", "INFO")
            print_status(f"  - 괄호 균형: {bracket_balanced}", "INFO")
        
        return validation_result
        
    except Exception as e:
        print_status(f"TTL 문법 검증 중 오류: {str(e)}", "ERROR")
        return False



def extract_ttl_from_response(message):
    try:
        if not getattr(message, "data", None):
            print_status("응답 메세지 없음", "ERROR")
            return None, False

        latest = message.data[0]
        if not latest.content:
            print_status("메세지 내용 없음", "ERROR")
            return None, False

        message_text = ""
        for item in latest.content:
            if hasattr(item, "text") and item.text and hasattr(item.text, "value"):
                message_text += item.text.value

        if not message_text.strip():
            print_status("텍스트 내용을 찾을 수 없습니다.", "ERROR")
            return None, False

        ttl_patterns = [
            r"```ttl\n(.*?)\n```",         # 1
            r"```turtle\n(.*?)\n```",      # 2
            r"```\n(.*?)\n```",            # 3
            r"```ttl\s*\n(.*?)\n```",      # 4
            r"```turtle\s*\n(.*?)\n```",   # 5
        ]

        PATTERN_EXPLANATIONS = {
            1: "```ttl 코드블록에서 추출됨",
            2: "```turtle 코드블록에서 추출됨",
            3: "라벨 없는 일반 ``` 코드블록에서 추출됨",
            4: "공백 포함 ```ttl 코드블록에서 추출됨",
            5: "공백 포함 ```turtle 코드블록에서 추출됨",
            0: "코드블록 미탐지 → 전체 응답을 사용하여 추출됨(폴백)",
        }

        ttl_content = None
        matched_idx = 0

        for i, pattern in enumerate(ttl_patterns, start=1):
            match = re.search(pattern, message_text, re.DOTALL | re.IGNORECASE)
            
            if match:
                ttl_content = match.group(1).strip()
                matched_idx = i
                break
                
        if not ttl_content:
            matched_idx = 0
            print_status("TTL 블록을 찾을 수 없습니다.", "ERROR")
        
        if len(ttl_content.strip()) < 50 :
            print_status("추출된 내용이 너무 짧습니다.", "ERROR")

        if not _validate_basic_ttl_syntax(ttl_content):
            print_status("TTL 문법을 확인해보세요.", "WARNING")


        print_status(f"TTL 추출 완료 (패턴 {matched_idx}) - {PATTERN_EXPLANATIONS[matched_idx]}", "SUCCESS" if matched_idx else "WARNING")
        return ttl_content, True

    except Exception as e:
        print_status(f"TTL 추출 중 오류가 발생했습니다.: {e}", "ERROR")


ontology_ttl_content = extract_ttl_from_response(ontology_message)

# 위에서 튜플 형태로 출력되기 떄문에 str로 변환해야 정상적으로 ttl 파일이 저장됨
# ('@prefix ex:     <http://example.org/disaster#> .\n@prefix rdfs: 
# ->  @prefix ex:     <http://example.org/disaster#> .
ontology_ttl_content_str = ontology_ttl_content[0]


[SUCCESS] TTL 추출 완료 (패턴 2) - ```turtle 코드블록에서 추출됨


### TTL 파일 생성 후 저장 O

In [62]:
def extract_filename(official_docs_path):
    """
    공공기관 문서명 추출
    """
    official_docs_name = os.path.basename(official_docs_path)
    official_docs_name_without_ext = os.path.splitext(official_docs_name)[0]
    return official_docs_name_without_ext

action_level_docs_name = extract_filename(actions_level_docs_path)

print(action_level_docs_name)

위기경보수준별조치사항


In [63]:
from datetime import datetime


def save_ttl_file(ttl_content, filename=None, output_dir="Ontology_Output"):
    try:
        
        # 저장할 디렉토리가 없을 경우 생성
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print_status(f"결과물을 저장할 폴더를 생성했습니다. : {output_dir}", "SUCCESS")

        # 파일명 설정
        if filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            filename = f"{filename}_Class_{timestamp}.ttl"

        # 파일 확장자 확인
        if not filename.endswith('.ttl'):
            filename += '.ttl'

        # 전체 파일 경로
        file_path = os.path.join(output_dir, filename)

        # 내용 검증
        if not ttl_content or len(ttl_content.strip()) < 50:
            return None, False, "저장할 TTL 내용이 유효하지 않습니다."

        with open(file_path, 'w', encoding='utf-8') as f:
            # 내용 저장
            f.write(ttl_content)

            # footer
            f.write("# End of TTL File\n")

        file_size = os.path.getsize(file_path)
        file_size_kb = file_size / 1024

        print_status("TTL 파일 생성 완료", "SUCCESS")
        print_status(f"{filename} 파일 경로: {file_path}", "INFO")
        print_status(f"{filename} 파일 크기: {file_size_kb:.2f}KB", "INFO")
        print_status(f"{filename} 내용 길이: {len(ttl_content)}자", "INFO")


    except Exception as e:
        print_status(f"TTL 파일 생성 실패: {e}", "ERROR")
        return None


final_output = save_ttl_file(ontology_ttl_content_str, action_level_docs_name)

[SUCCESS] TTL 파일 생성 완료
[INFO] 위기경보수준별조치사항_Class_20250905_1808.ttl 파일 경로: Ontology_Output\위기경보수준별조치사항_Class_20250905_1808.ttl
[INFO] 위기경보수준별조치사항_Class_20250905_1808.ttl 파일 크기: 5.15KB
[INFO] 위기경보수준별조치사항_Class_20250905_1808.ttl 내용 길이: 3285자


--------

--------

# ** STEP 2 : 페이지 분리 후 Subclass 보완 추가 ** 

In [97]:
def run_assistant_page_json(client, thread_id, addundum_page_json_prompt):

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions= addundum_page_json_prompt
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == 'completed':
            print_status("Assistant 실행 완료", "SUCCESS")
            break
        
        elif run_status.status == 'failed':
            print_status("Assistant 실행 실패", "ERROR")

        elif run_status.status == 'requires_action':
            print_status("추가 액션 필요", "WARNING")
        
        elif run_status.status == 'in_progress':
            print_status("Assistant 실행 중...", "INFO")

            time.sleep(10)

    page_json_message = client.beta.threads.messages.list(
        thread_id=thread_id,
        order="desc"
    )

    return page_json_message

In [99]:
page_json_thread_id = create_thread(client)
page_json_result = run_assistant_page_json(client, page_json_thread_id, addundum_page_json_prompt)
print(page_json_result)

C:\Users\user\AppData\Local\Temp\ipykernel_55848\103296605.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


[SUCCESS] 스레드 생성 완료: thread_TQXHURyitvHF4kFPSAj1vAA1


C:\Users\user\AppData\Local\Temp\ipykernel_55848\3738730227.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\user\AppData\Local\Temp\ipykernel_55848\3738730227.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


[INFO] Assistant 실행 중...
[SUCCESS] Assistant 실행 완료


C:\Users\user\AppData\Local\Temp\ipykernel_55848\3738730227.py:30: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  page_json_message = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_ho0yd0u83KkSIhbqBPBFhcKq', assistant_id='asst_W7kaWMqWblyhhVj1rCP0mVow', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="업로드하신 파일에서 '문단 단위의 구조'와 'section_uid' 등 목차 관련 정보를 찾기 위해 다시 시도할 준비가 되어 있습니다.  \n파일 내용을 구체적으로 보면서 페이지별 문단 단위로 구조를 분석해야 하는데, 검색 도구에서 오류가 발생해 직접 파일 내용 요약 또는 명확한 지침이 필요합니다.  \n\n분석할 PDF 문서의 목차나 본문 구조 일부를 제시해 주시면, 그 부분을 기준으로 페이지별 문단 구성과 section_uid를 체계적으로 추출하여 JSON 형식으로 출력해 드리겠습니다.  \n원하시는 특정 범위나 예시 페이지가 있으면 알려주시기 바랍니다."), type='text')], created_at=1757065234, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_pp9zts7aH1mKzuG3WriG2lnj', status=None, thread_id='thread_TQXHURyitvHF4kFPSAj1vAA1')], has_more=False, object='list', first_id='msg_ho0yd0u83KkSIhbqBPBFhcKq', last_id='msg_ho0yd0u83KkSIhbqBPBFhcKq')


In [ ]:
# page_json_result를 가지고 서브 클래스 보완하는 프롬프트 작성해서 어시스턴트한테 시키기
add_subclass_message = run_assistant_with_task_prompt(client, thread_id, base_prompt, add_subclass_prompt)
print([str(ontology_message)][:10])


In [ ]:
from datetime import datetime


def save_ttl_file(ttl_content, filename=None, output_dir="Ontology_Output"):
    try:
        
        # 저장할 디렉토리가 없을 경우 생성
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print_status(f"결과물을 저장할 폴더를 생성했습니다. : {output_dir}", "SUCCESS")

        # 파일명 설정
        if filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            filename = f"{filename}_Class_{timestamp}.ttl"

        # 파일 확장자 확인
        if not filename.endswith('.ttl'):
            filename += '.ttl'

        # 전체 파일 경로
        file_path = os.path.join(output_dir, filename)

        # 내용 검증
        if not ttl_content or len(ttl_content.strip()) < 50:
            return None, False, "저장할 TTL 내용이 유효하지 않습니다."

        with open(file_path, 'w', encoding='utf-8') as f:
            # 내용 저장
            f.write(ttl_content)

            # footer
            f.write("# End of TTL File\n")

        file_size = os.path.getsize(file_path)
        file_size_kb = file_size / 1024

        print_status("TTL 파일 생성 완료", "SUCCESS")
        print_status(f"{filename} 파일 경로: {file_path}", "INFO")
        print_status(f"{filename} 파일 크기: {file_size_kb:.2f}KB", "INFO")
        print_status(f"{filename} 내용 길이: {len(ttl_content)}자", "INFO")


    except Exception as e:
        print_status(f"TTL 파일 생성 실패: {e}", "ERROR")
        return None


final_output = save_ttl_file(ontology_ttl_content_str, action_level_docs_name)